In [2]:
import glob

f2BCMB=glob.glob('/Volumes/T7 Shield/SUBSETS/Subsets/2B-CS-CONUS*')
f2BCMB=sorted(f2BCMB)
f2ADPR=glob.glob('/Volumes/T7 Shield/SUBSETS/Subsets/2A-CS-CONUS*DPR*')
f2ADPR=sorted(f2ADPR)
print(len(f2BCMB),len(f2ADPR))
import readStormData

658 653


In [10]:
ifile=0
import numpy as np
bzdL=[]
bcfL=[]
bsfcL=[]
zMeasL=[]
pRateL=[]
pTypeL=[]
pFlagL=[]
reliabFL=[]
zFactorFL=[]
PIAhybridL=[]
dmL=[]

import netCDF4 as nc
jL=[]
for f in f2BCMB[:300]:
    ifile+=1
    orb=f.split('.')[-3]
    for f2 in f2ADPR:
        if orb in f2:
            break
    if orb not in f2:
        print('No match for ',orb)
        continue
    with nc.Dataset(f2) as fDPR:
        pType=fDPR["FS/CSF/typePrecip"][:]
        pType=(pType/1e7).astype(int)
        a=np.nonzero(pType==1)
    zMeas,bcf,bsfc,bzd,pFlag,reliabF,pType,zFactorF,PIAhybrid,dm,pRate=readStormData.readStormDataf(f,f2)
    for i,j in zip(a[0],a[1]):
        if bzd[i,j]+14<bcf[i,j]:
            continue
        if j<12 or j>37:
            continue
        bzdL.append(bzd[i,j])
        bcfL.append(bcf[i,j])
        bsfcL.append(bsfc[i,j])
        zMeasL.append(zMeas[i,j])
        pRateL.append(pRate[i,j])
        pTypeL.append(pType[i,j])
        pFlagL.append(pFlag[i,j])
        reliabFL.append(reliabF[i,j])
        zFactorFL.append(zFactorF[i,j])
        PIAhybridL.append(PIAhybrid[i,j])
        dmL.append(dm[i,j])
        jL.append(j)


    #break
            


In [11]:
print(bzdL[0])
print(bcfL[0])
print(bsfcL[0])
print(len(bzdL))

151
164
[172 172]
109675


In [12]:
import xarray as xr
bzdX=xr.DataArray(bzdL,dims=['n'])
bcfX=xr.DataArray(bcfL,dims=['n'])
bsfcX=xr.DataArray(bsfcL,dims=['n','n2'])
zMeasX=xr.DataArray(zMeasL,dims=['n','nbins','nch'])
pRateX=xr.DataArray(pRateL,dims=['n','nbins2'])
pTypeX=xr.DataArray(pTypeL,dims=['n'])
pFlagX=xr.DataArray(pFlagL,dims=['n'])
reliabFX=xr.DataArray(reliabFL,dims=['n'])
zFactorFX=xr.DataArray(zFactorFL,dims=['n','nbins','nch'])
PIAhybridX=xr.DataArray(PIAhybridL,dims=['n','nch'])
jLX=xr.DataArray(jL,dims=['n'])

ds=xr.Dataset({'bzd':bzdX,'bcf':bcfX,'bsfc':bsfcX,'zMeas':zMeasX,'pRate':pRateX,'pType':pTypeX,'pFlag':pFlagX,'reliabF':reliabFX,'zFactorF':zFactorFX,'PIAhybrid':PIAhybridX,'j':jLX})
complev=5
encoding={'bzd':{'zlib':True,'complevel':complev},'bcf':{'zlib':True,'complevel':complev},'bsfc':{'zlib':True,'complevel':complev},'zMeas':{'zlib':True,'complevel':complev},'pRate':{'zlib':True,'complevel':complev},'pType':{'zlib':True,'complevel':complev},'pFlag':{'zlib':True,'complevel':complev},'reliabF':{'zlib':True,'complevel':complev},'zFactorF':{'zlib':True,'complevel':complev},'PIAhybrid':{'zlib':True,'complevel':complev},'j':{'zlib':True,'complevel':complev}}
ds.to_netcdf('stratProfs.nc',encoding=encoding)
